In [ ]:
import pandas as pd
import dask as dask
from dask import dataframe as dd
import numpy as np
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask.array as da
import functools
import sys
import ast
from struct import *
import pickle
import glob
import random
import seaborn as snsa
import ipywidgets as widgets
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
file = open('instance_files/instance_3radii_iceagg_allrand_100xtalstot_20phi_r10_1000agg', 'rb')
b1 = pickle.load(file)
file = open('instance_files/instance_3radii_iceagg_allflat_100xtalstot_20phi_r10_1000agg', 'rb')
b2 = pickle.load(file)

In [ ]:
phiba_rand = b1.rxs/b1.rzs
phica_rand = b1.rys/b1.rzs
phiba_flat = b2.rxs/b2.rzs
phica_flat = b2.rys/b2.rzs

In [ ]:
n, bins, patches = plt.hist(phiba[10,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(phica[10,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand plates
n, bins, patches = plt.hist(b1.rxs[1,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand columns 
n, bins, patches = plt.hist(b1.rxs[19,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[19,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[19,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
n, bins, patches = plt.hist(b1.rxs[1,:,19], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,19], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,19], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
n, bins, patches = plt.hist(a[10,:,19], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b[10,:,19], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(c[10,:,19], bins='auto', color='b',
                            alpha=0.7, rwidth=0.85)


In [ ]:
a = np.zeros((20,1000,98))
b = np.zeros((20,1000,98))
c = np.zeros((20,1000,98))

for phi in np.arange(20):
    print(phi)
    for i in np.arange(1000):
        for N in np.arange(98):
            a[phi,i,N], b[phi,i,N], c[phi,i,N] = sorted([b1.rxs[phi,i,N], b1.rys[phi,i,N], b1.rzs[phi,i,N]], reverse=True)
            '''
            a[phi,i,N] = max(b1.rxs[phi,i,N], b1.rys[phi,i,N], b1.rzs[phi,i,N])
            c[phi,i,N] = min(b1.rxs[phi,i,N], b1.rys[phi,i,N], b1.rzs[phi,i,N])
            if b1.rxs[phi,i,N] != a[phi,i,N] and b1.rxs[phi,i,N] != c[phi,i,N]:
                b[phi,i,N] = b1.rxs[phi,i,N]
                
            elif b1.rys[phi,i,N] != a[phi,i,N] and b1.rys[phi,i,N] != c[phi,i,N]:        
                b[phi,i,N] = b1.rys[phi,i,N]
                
            else:   
                b[phi,i,N] = b1.rzs[phi,i,N]
            '''
              
print(np.shape(b))

In [ ]:
#random orientation, ice-agg
#a_ellipse (rzs) to a_mono (length for columns, width for plates)
phios = [8, 11, 5, 14,10]
Ns = np.arange(0,98,1)
Nslab = np.arange(2, 100, 1)
for phio in phios:
    ellipse_ratio=[]
    for N in Ns:
        if phio <= 10.0: #plates
            ellipse_ratio.append(np.mean(a[phio,:,N])/np.mean(b1.width[phio]))
        else: #columns
            ellipse_ratio.append(np.mean(a[phio,:,N])/np.mean(b1.length[phio]))

    if ('%.2f')%b1.phio[phio] == str(1.27): 
        label='spheres'
        color = 'k'
        linestyle='-'
    if ('%.2f')%b1.phio[phio] == str(0.48):
        label='plates'
        color = 'b'
        linestyle=None
        zorder = 2
    if ('%.2f')%b1.phio[phio] == str(2.07):
        label='columns'
        color='r'
        linestyle=None
    if ('%.2f')%b1.phio[phio] == str(0.11):
        label = 'plates'
        color = 'b'
        linestyle = '--'
    if ('%.2f')%b1.phio[phio] == str(8.86):
        label='columns'
        color='r'
        linestyle='--'
    
    
    plt.plot(Nslab, ellipse_ratio, label=label, color=color, linestyle=linestyle)
    plt.fill_between(Nslab, ellipse_ratio-np.std(ellipse_ratio), ellipse_ratio+np.std(ellipse_ratio), alpha = 0.1, color=color)
    plt.ylim(0,16)
    plt.xlim(0,100)
    plt.ylabel('$a_{ellip}/a_{mon}$', fontsize = 20)
    plt.xlabel('N', fontsize=20)
    plt.legend()

    
#columns and plates are not initialized with the same axis lengths in order 
#to keep the equivalent volume radius constant

#within columns and plates, we are dividing my the same length magnitudes for a_mon

#dashed lines dividing by a larger a_mon, but also scales faster with N since longer

In [ ]:
phios = [3, 14, 10]
Ns = np.arange(0,98,1)
Nslab = np.arange(2, 100, 1)
for phio in phios:
    phibaplt=[]
    phicaplt=[]
    print(('%.2f')%b1.phio[phio])
    for N in Ns:
        phibaplt.append(np.mean(b[phio,:,N]/a[phio,:,N]))
       
        phicaplt.append(np.mean(c[phio,:,N]/a[phio,:,N]))
    
    if ('%.2f')%b1.phio[phio] == str(1.27): 
        label='spheres'
        color = 'k'
        linestyle='-'
    if ('%.2f')%b1.phio[phio] == str(0.04):
        label='plates'
        color = 'b'
        linestyle=None
        zorder = 2
    if ('%.2f')%b1.phio[phio] == str(2.07):
        label='columns'
        color='r'
        linestyle=None
    if ('%.2f')%b1.phio[phio] == str(0.11):
        label = 'plates'
        color = 'b'
        linestyle = '--'
    if ('%.2f')%b1.phio[phio] == str(8.86):
        label='columns'
        color='r'
        linestyle='--'
    
    plt.plot(Nslab, phibaplt, color=color, linestyle=None)
    plt.plot(Nslab, phicaplt, label=label, color=color, linestyle='--')
    
    plt.fill_between(Nslab, phibaplt-np.std(phibaplt), phibaplt+np.std(phibaplt), alpha = 0.1, color=color)
    plt.fill_between(Nslab, phicaplt-np.std(phicaplt), phicaplt+np.std(phicaplt), alpha = 0.1, color=color)

plt.ylim(0,1.0)
plt.xlim(2,100)
plt.ylabel('$\phi$', fontsize = 20)
plt.title('$\phi_{ba} (solid) \phi_{ca} (dashed)$', fontsize = 20)
plt.xlabel('$N_{mon}$', fontsize=20)
plt.legend()

In [ ]:
#rhoio = Vagg/Vio
phios = [3, 14, 10]
Ns = np.arange(0,98,1)
Nslab = np.arange(2, 100, 1)
for phio in phios:
    rhoio=[]
    print(('%.2f')%b1.phio[phio])
    for N in Ns:
        #rhoio.append(np.mean((4./3.)*np.pi*a[phio,:,N]*b[phio,:,N]*c[phio,:,N]/((4./3.)*np.pi*a[phio,:,N]**3.)))
        if phio < ..
        
            rhoio.append(np.mean((3.*np.sqrt(3.)/2.)*np.power(a[phio,:,N]*b[phio,:,N]*c[phio,:,N]/((4./3.)*a[phio,:,N]*b[phio,:,N]*c[phio,:,N])))
        else:
            
    if ('%.2f')%b1.phio[phio] == str(1.27): 
        label='spheres'
        color = 'k'
        linestyle='-'
    if ('%.2f')%b1.phio[phio] == str(0.04):
        label='plates'
        color = 'b'
        linestyle=None
        zorder = 2
    if ('%.2f')%b1.phio[phio] == str(2.07):
        label='columns'
        color='r'
        linestyle=None
    if ('%.2f')%b1.phio[phio] == str(0.11):
        label = 'plates'
        color = 'b'
        linestyle = '--'
    if ('%.2f')%b1.phio[phio] == str(8.86):
        label='columns'
        color='r'
        linestyle='--'
    
    plt.plot(Nslab, rhoio, label=label, color=color, linestyle='-')
    
    plt.fill_between(Nslab, rhoio-np.std(rhoio), rhoio+np.std(rhoio), alpha = 0.1, color=color)

#plt.ylim(0,1.0)
plt.xlim(2,100)
plt.ylabel('$V_{agg}/V_{io}$', fontsize = 20)
plt.xlabel('$N_{mon}$', fontsize=20)
plt.legend()
#Vagg = (3.*np.sqrt(3.)/2)* np.power(b1.width, 2)*b1.length*100